In [1]:
from argparse import ArgumentParser
from pathlib import Path
from sklearn.model_selection import *
import matplotlib.pyplot as plt
from sklearn.metrics import *
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.layers import InputLayer, Conv1D, BatchNormalization, ReLU, Dense, Flatten, Softmax, LSTM
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Reshape
from tensorflow.keras.layers import *
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from scipy.signal import resample
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import mode
import os
from tensorflow.keras.models import Model

In [2]:
data_path = 'data/'
data_train = pd.read_csv(data_path+'sensor_train.csv')
data_test = pd.read_csv(data_path+'sensor_test.csv')
data_test['fragment_id'] += 10000
label = 'behavior_id'
y_train = data_train.groupby('fragment_id')['behavior_id'].min().values

data = pd.concat([data_train, data_test], sort=False)
df = data.drop_duplicates(subset=['fragment_id']).reset_index(drop=True)[['fragment_id', 'behavior_id']]

In [3]:
def load_features_data(data, df):
    # 加速度绝对值
    data['acc'] = (data['acc_x'] ** 2 + data['acc_y'] ** 2 + data['acc_z'] ** 2) ** 0.5
    data['accg'] = (data['acc_xg'] ** 2 + data['acc_yg'] ** 2 + data['acc_zg'] ** 2) ** 0.5
    
    data['accxy'] = (data['acc_x'] ** 2 + data['acc_y'] ** 2) ** 0.5
    data['accyz'] = (data['acc_y'] ** 2 + data['acc_z'] ** 2) ** 0.5
    data['accxz'] = (data['acc_x'] ** 2 + data['acc_z'] ** 2) ** 0.5
    data['accxyg'] = (data['acc_xg'] ** 2 + data['acc_yg'] ** 2) ** 0.5
    data['accyzg'] = (data['acc_yg'] ** 2 + data['acc_zg'] ** 2) ** 0.5
    data['accxzg'] = (data['acc_xg'] ** 2 + data['acc_zg'] ** 2) ** 0.5
    
#     data['acc_sub'] = ((data['acc_xg'] - data['acc_x']) ** 2 + (data['acc_yg'] - data['acc_y']) ** 2 + (data['acc_zg'] - data['acc_z']) ** 2) ** 0.5
#     data['acc_subxy'] = ((data['acc_xg'] - data['acc_x']) ** 2 + (data['acc_yg'] - data['acc_y']) ** 2) ** 0.5
#     data['acc_subxz'] = ((data['acc_xg'] - data['acc_x']) ** 2 + (data['acc_zg'] - data['acc_z']) ** 2) ** 0.5
#     data['acc_subyz'] = ((data['acc_yg'] - data['acc_y']) ** 2 + (data['acc_zg'] - data['acc_z']) ** 2) ** 0.5
    
    # 统计特征
    for f in tqdm([f for f in data.columns if 'acc' in f]):
        for stat in ['min', 'mean', 'median', 'std']:   # skew
            df['{}_{}'.format(f, stat)] = data.groupby('fragment_id')[f].agg(stat).values
    
    train_df = df[df[label].isna()==False].reset_index(drop=True)
    test_df = df[df[label].isna()==True].reset_index(drop=True)
    
    drop_feat = [] 
    used_feat = [f for f in train_df.columns if f not in (['fragment_id', label] + drop_feat)] 
    print('特征个数 ', len(used_feat))
    print(used_feat)
    
    X_train = train_df[used_feat].values
    y_train = train_df[label].values
    X_test = test_df[used_feat].values

    return X_train, y_train, X_test


def load_y(df):
    return y_train


In [4]:
train_features, _, test_features = load_features_data(data, df)
y = load_y(df)

100%|██████████| 17/17 [00:02<00:00,  8.09it/s]

特征个数  68
['acc_x_min', 'acc_x_mean', 'acc_x_median', 'acc_x_std', 'acc_y_min', 'acc_y_mean', 'acc_y_median', 'acc_y_std', 'acc_z_min', 'acc_z_mean', 'acc_z_median', 'acc_z_std', 'acc_xg_min', 'acc_xg_mean', 'acc_xg_median', 'acc_xg_std', 'acc_yg_min', 'acc_yg_mean', 'acc_yg_median', 'acc_yg_std', 'acc_zg_min', 'acc_zg_mean', 'acc_zg_median', 'acc_zg_std', 'acc_min', 'acc_mean', 'acc_median', 'acc_std', 'accg_min', 'accg_mean', 'accg_median', 'accg_std', 'accxy_min', 'accxy_mean', 'accxy_median', 'accxy_std', 'accyz_min', 'accyz_mean', 'accyz_median', 'accyz_std', 'accxz_min', 'accxz_mean', 'accxz_median', 'accxz_std', 'accxyg_min', 'accxyg_mean', 'accxyg_median', 'accxyg_std', 'accyzg_min', 'accyzg_mean', 'accyzg_median', 'accyzg_std', 'accxzg_min', 'accxzg_mean', 'accxzg_median', 'accxzg_std', 'acc_subxy_min', 'acc_subxy_mean', 'acc_subxy_median', 'acc_subxy_std', 'acc_subxz_min', 'acc_subxz_mean', 'acc_subxz_median', 'acc_subxz_std', 'acc_subyz_min', 'acc_subyz_mean', 'acc_subyz_medi

In [5]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# train_features = scaler.fit_transform(train_features)
# test_features  = scaler.fit_transform(test_features)

In [6]:
# def Net(type='dense'):
#     fea_input = Input(shape=(train_features.shape[1]))

#     dense = Dense(32, activation='relu')(fea_input)
#     dense = BatchNormalization()(dense)
#     dense = Dense(64, activation='relu')(dense)
#     dense = Dropout(0.2)(dense)
#     dense = Dense(128, activation='relu')(dense)
#     dense = Dropout(0.2)(dense)
#     dense = Dense(256, activation='relu')(dense)
#     dense = Dropout(0.3)(dense)

#     dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
#     X = Dense(19, activation='softmax')(dense)
#     return Model([fea_input], X)


# def Net(type='dense'):
#     fea_input = Input(shape=(train_features.shape[1]))

#     dense = Dense(32, activation='relu')(fea_input)
#     dense = BatchNormalization()(dense)
#     dense = Dense(64, activation='relu')(dense)
#     dense = Dropout(0.1)(dense)
#     dense = Dense(128, activation='relu')(dense)
#     dense = Dropout(0.1)(dense)
#     dense = Dense(256, activation='relu')(dense)
#     dense = Dropout(0.2)(dense)
#     dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
#     X = Dense(19, activation='softmax')(dense)
#     return Model([fea_input], X)

# def Net(type='dense'):
#     fea_input = Input(shape=(train_features.shape[1]))

#     dense = Dense(32, activation='relu')(fea_input)
#     dense = BatchNormalization()(dense)
#     dense = Dense(64, activation='relu')(dense)
# #     dense = Dropout(0.1)(dense)
#     dense = Dense(128, activation='relu')(dense)
# #     dense = Dropout(0.1)(dense)
#     dense = Dense(256, activation='relu')(dense)
# #     dense = Dropout(0.1)(dense)
#     dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
#     X = Dense(19, activation='softmax')(dense)
#     return Model([fea_input], X)

# def Net(type='dense'):
#     fea_input = Input(shape=(train_features.shape[1]))

#     dense = Dense(32, activation='relu')(fea_input)
#     dense = BatchNormalization()(dense)
#     dense = Dense(64, activation='relu')(dense)
# #     dense = Dropout(0.1)(dense)
#     dense = Dense(128, activation='relu')(dense)
# #     dense = Dropout(0.1)(dense)
#     dense = Dense(256, activation='relu')(dense)
# #     dense = Dropout(0.1)(dense)
#     dense = Dense(512, activation='relu')(dense)
#     dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
#     X = Dense(19, activation='softmax')(dense)
#     return Model([fea_input], X)

# def Net(type='dense'):
#     fea_input = Input(shape=(train_features.shape[1]))
    
#     dense = Dense(64, activation='relu')(fea_input)
#     dense = BatchNormalization()(dense)
#     dense = Dense(128, activation='relu')(dense)
#     dense = Dense(256, activation='relu')(dense)
#     dense = Dense(512, activation='relu')(dense)
#     dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
#     X = Dense(19, activation='softmax')(dense)
#     return Model([fea_input], X)

# def Net(type='dense'):
#     fea_input = Input(shape=(train_features.shape[1]))
    
#     dense = Dense(128, activation='relu')(fea_input)
#     dense = BatchNormalization()(dense)
# #     dense = Dense(128, activation='relu')(dense)
#     dense = Dense(256, activation='relu')(dense)
#     dense = Dense(512, activation='relu')(dense)
#     dense = Dense(1024, activation='relu')(dense)
#     dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
#     X = Dense(19, activation='softmax')(dense)
#     return Model([fea_input], X)


# def Net(type='dense'):
#     fea_input = Input(shape=(train_features.shape[1]))
    
#     dense = Dense(128, activation='relu')(fea_input)
#     dense = BatchNormalization()(dense)
# #     dense = Dense(128, activation='relu')(dense)
#     dense = Dense(256, activation='relu')(dense)
#     dense = Dense(512, activation='relu')(dense)
# #     dense = Dense(1024, activation='relu')(dense)
#     dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
#     X = Dense(19, activation='softmax')(dense)
#     return Model([fea_input], X)

# def Net(type='dense'):
#     fea_input = Input(shape=(train_features.shape[1]))
    
#     dense = Dense(256, activation='relu')(fea_input)
#     dense = BatchNormalization()(dense)
#     dense = Dense(512, activation='relu')(dense)
#     dense = Dense(1024, activation='relu')(dense)
#     dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
#     X = Dense(19, activation='softmax')(dense)
#     return Model([fea_input], X)

def Net(type='dense'):
    fea_input = Input(shape=(train_features.shape[1]))
    
    dense = Dense(256, activation='relu')(fea_input)
    dense = BatchNormalization()(dense)
    dense = Dense(512, activation='relu')(dense)
    dense = Dense(1024, activation='relu')(dense)
    dense = BatchNormalization()(Dropout(0.2)(Dense(64, activation='relu')(Flatten()(dense))))
    X = Dense(19, activation='softmax')(dense)
    return Model([fea_input], X)


In [7]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0

In [ ]:
# acc_scores = []
# combo_scores = []
# proba_xx = np.zeros((7292, 19))
# proba_t = np.zeros((7500, 19))
# kfold = StratifiedKFold(5, shuffle=True, random_state=42)

# for fold, (train_index, valid_index) in enumerate(kfold.split(train_features, y)):
#     print("{}train {}th fold{}".format('==' * 20, fold + 1, '==' * 20))
#     y_ = to_categorical(y, num_classes=19)
#     model = Net(type='conv1')
#     model.compile(loss='categorical_crossentropy',
#                   optimizer='rmsprop',
#                   metrics=['acc'])
#     model.summary()
#     plateau = ReduceLROnPlateau(monitor="val_acc",
#                                 verbose=1,
#                                 mode='max',
#                                 factor=0.5,
#                                 patience=20)
#     early_stopping = EarlyStopping(monitor='val_acc',
#                                    verbose=1,
#                                    mode='max',
#                                    patience=40)
#     checkpoint = ModelCheckpoint(f'models/fold{fold}.h5',
#                                  monitor='val_acc',
#                                  verbose=0,
#                                  mode='max',
#                                  save_best_only=True)

#     csv_logger = CSVLogger('logs/log.csv', separator=',', append=True)
#     history = model.fit(train_features[train_index], y_[train_index],
#                         epochs=500,
#                         batch_size=32,
#                         verbose=2,
#                         shuffle=True,
#                         validation_data=(train_features[valid_index],
#                                          y_[valid_index]),
#                         callbacks=[plateau, early_stopping, checkpoint, csv_logger])
#     model.load_weights(f'models/fold{fold}.h5')
#     proba_x = model.predict(train_features[valid_index], verbose=0, batch_size=1024)
#     proba_xx[valid_index] = proba_x
#     proba_t += model.predict(test_features, verbose=0, batch_size=1024) / 5.

#     oof_y = np.argmax(proba_x, axis=1)
#     score1 = accuracy_score(y[valid_index], oof_y)
#     # print('accuracy_score',score1)
#     score = sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y[valid_index], oof_y)) / oof_y.shape[0]
#     print('accuracy_score', score1, 'acc_combo', score)
#     acc_scores.append(score1)
#     combo_scores.append(score)

# #     print(history.history.keys())
# #     # summarize history for accuracy
# #     plt.plot(history.history['acc'])
# #     plt.plot(history.history['val_acc'])
# #     plt.title('model accuracy')
# #     plt.ylabel('accuracy')
# #     plt.xlabel('epoch')
# #     plt.legend(['train', 'test'], loc='upper left')
# #     plt.show()
# #     # summarize history for loss
# #     plt.plot(history.history['loss'])
# #     plt.plot(history.history['val_loss'])
# #     plt.title('model loss')
# #     plt.ylabel('loss')
# #     plt.xlabel('epoch')
# #     plt.legend(['train', 'test'], loc='upper left')
# #     plt.show()

# print("5kflod mean acc score:{}".format(np.mean(acc_scores)))
# print("5kflod mean combo score:{}".format(np.mean(combo_scores)))


acc_scores = []
combo_scores = []
proba_xx = np.zeros((7292, 19))
proba_t = np.zeros((7500, 19))
kfold = StratifiedKFold(5, shuffle=True, random_state=42)

for fold, (train_index, valid_index) in enumerate(kfold.split(train_features, y)):
    print("{}train {}th fold{}".format('==' * 20, fold + 1, '==' * 20))
    y_ = to_categorical(y, num_classes=19)
    model = Net(type='conv1')
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    model.summary()
    plateau = ReduceLROnPlateau(monitor="val_acc",
                                verbose=1,
                                mode='max',
                                factor=0.5,
                                patience=20)
    early_stopping = EarlyStopping(monitor='val_acc',
                                   verbose=1,
                                   mode='max',
                                   patience=40)
    checkpoint = ModelCheckpoint(f'models/fold{fold}.h5',
                                 monitor='val_acc',
                                 verbose=0,
                                 mode='max',
                                 save_best_only=True)

    csv_logger = CSVLogger('logs/log.csv', separator=',', append=True)
    history = model.fit(train_features[train_index], y_[train_index],
                        epochs=500,
                        batch_size=32,
                        verbose=2,
                        shuffle=True,
                        validation_data=(train_features[valid_index],
                                         y_[valid_index]),
                        callbacks=[plateau, early_stopping, checkpoint, csv_logger])
    model.load_weights(f'models/fold{fold}.h5')
    proba_x = model.predict(train_features[valid_index], verbose=0, batch_size=64)
    proba_xx[valid_index] = proba_x
    proba_t += model.predict(test_features, verbose=0, batch_size=64) / 5.

    oof_y = np.argmax(proba_x, axis=1)
    score1 = accuracy_score(y[valid_index], oof_y)
    # print('accuracy_score',score1)
    score = sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y[valid_index], oof_y)) / oof_y.shape[0]
    print('accuracy_score', score1, 'acc_combo', score)
    acc_scores.append(score1)
    combo_scores.append(score)

print("5kflod mean acc score:{}".format(np.mean(acc_scores)))
print("5kflod mean combo score:{}".format(np.mean(combo_scores)))

========================================train 1th fold========================================
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 68)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               17664     
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
________________________________

In [ ]:
sub = pd.read_csv('data/提交结果示例.csv')
sub.behavior_id = np.argmax(proba_t, axis=1)
sub.to_csv('result/dnn_acc{}_combo{}.csv'.format(round(np.mean(acc_scores), 5), round(np.mean(combo_scores), 5)), index=False)


# acc 654 up 668
# 5kflod mean acc score:0.6907564889795235  691
# 5kflod mean acc score:0.7309383787869814  714
# 5kflod mean acc score:0.747531569342551   725
# 5kflod mean acc score:0.7619320409435404  727

In [ ]:
pd.DataFrame(proba_t, columns=['pred_{}'.format(i) for i in range(19)]).to_csv('result/har_proba_t_{}.csv'.format(round(np.mean(acc_scores), 5)), index=False)
pd.DataFrame(proba_xx, columns=['pred_{}'.format(i) for i in range(19)]).to_csv('result/har_proba_x_{}.csv'.format(round(np.mean(acc_scores), 5)), index=False)


In [ ]:
np.shape(proba_xx)